In [ ]:
import numpy as np
import pandapower as pp
from deap import base, creator, tools, algorithms
import random

# === IEEE 14-bus 시스템 데이터 템플릿 ===
bus_data_template = np.array([
    [1,  0.0,        0.0,       1.05, 0.0],
    [2, -21.7/100,   0.0,       1.05,  None],
    [2, -94.2/100,   0.0,       1.05,  None],
    [3, -47.8/100,   -(-3.9)/100, None, None],
    [3, -7.6/100,    -0.6/100,    None, None],
    [2, -11.2/100,   0.0,       1.05,  None],
    [3,  0.0,        0.0,       None, None],
    [3,  0.0,        0.0,       None, None],
    [3, -29.5/100,   -3.6/100,    None, None],
    [3, -9/100,      -1.5/100,    None, None],
    [3, -3.5/100,    -0.1/100,    None, None],
    [3, -6.1/100,    -0.6/100,    None, None],
    # Ppu 값은 GA 변수로 설정됨
    [3, None,       -2.8/100,    None, None],  # bus 13
    [3, None,       -1.0/100,    None, None],  # bus 14
], dtype=object)

branch_data = np.array([
    [1, 2, 0.01938, 0.05917, 0.0528],
    [1, 5, 0.05403, 0.22304, 0.0492],
    [2, 3, 0.04699, 0.19797, 0.0438],
    [2, 4, 0.05811, 0.17632, 0.0340],
    [2, 5, 0.05695, 0.17388, 0.0346],
    [3, 4, 0.06701, 0.17103, 0.0128],
    [4, 5, 0.01335, 0.04211, 0.0   ],
    [4, 7, 0.0,     0.20912, 0.0   ],
    [4, 9, 0.0,     0.55618, 0.0   ],
    [5, 6, 0.0,     0.25202, 0.0   ],
    [6,11, 0.09498, 0.19890, 0.0   ],
    [6,12, 0.12291, 0.25581, 0.0   ],
    [6,13, 0.06615, 0.13027, 0.0   ],
    [7, 8, 0.0,     0.17615, 0.0   ],
    [7, 9, 0.0,     0.11001, 0.0   ],
    [9,10, 0.03181, 0.08450, 0.0   ],
    [9,14, 0.12711, 0.27038, 0.0   ],
    [10,11,0.08205, 0.19207, 0.0   ],
    [12,13,0.22092, 0.19988, 0.0   ],
    [13,14,0.17093, 0.34802, 0.0   ],
], dtype=object)

# 기본 단위
S_base = 100      # MVA
V_base_kv = 110   # kV
Z_base = V_base_kv**2 / S_base

# 네트워크 생성 함수
def build_network(x0, x1):
    bus_data = bus_data_template.copy()
    bus_data[12,1] = -x0/100
    bus_data[13,1] = -x1/100

    net = pp.create_empty_network()
    for i, b in enumerate(bus_data):
        pp.create_bus(net, vn_kv=V_base_kv, name=f"Bus {i+1}")
    pp.create_ext_grid(net, bus=0,
                       vm_pu=bus_data[0,3],
                       va_degree=bus_data[0,4])
    for i, b in enumerate(bus_data):
        typ, Ppu, Qpu, Vsp, _ = b
        if typ == 2:
            Pmw = -Ppu * S_base
            pp.create_gen(net, bus=i, p_mw=Pmw, vm_pu=Vsp,
                          min_q_mvar=-50, max_q_mvar=50)
        elif typ == 3:
            Pmw = -Ppu * S_base
            Qmvar = -Qpu * S_base
            pp.create_load(net, bus=i, p_mw=Pmw, q_mvar=Qmvar)
    for f, t, R, X, Bpu in branch_data:
        r_ohm = R * Z_base
        x_ohm = X * Z_base
        b_actual = Bpu / Z_base
        c_nf_per_km = b_actual/(2*np.pi*50)*1e9
        pp.create_line_from_parameters(net,
            from_bus=int(f-1), to_bus=int(t-1), length_km=1.0,
            r_ohm_per_km=r_ohm, x_ohm_per_km=x_ohm,
            c_nf_per_km=c_nf_per_km, max_i_ka=1.0)
    return net

# GA 설정
import deap.creator as creator
# 기존 정의 삭제 (Jupyter 셀 재실행 시 대비)
if "FitnessMax" in creator.__dict__:
    del creator.FitnessMax
if "Individual" in creator.__dict__:
    del creator.Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_var", random.uniform, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_var, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(ind):
    x0, x1 = ind
    net = build_network(x0, x1)
    try:
        pp.runpp(net)
    except Exception:
        return (-1000,)
    vm = net.res_bus.vm_pu.values
    if np.any(vm < 1.0) or np.any(vm > 1.05):
        return (-1000,)
    return (x0 + x1,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 실행 함수
def main():
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)
    stats.register("avg", np.mean)
    pop, log = algorithms.eaSimple(pop, toolbox,
                                   cxpb=0.5, mutpb=0.2,
                                   ngen=50, stats=stats,
                                   halloffame=hof, verbose=True)
    best_x0, best_x1 = hof[0]
    print("Best solution:", hof[0], "Sum(x):", hof[0].fitness.values[0])

    # 최적해로 네트워크 재구성 및 부하흐름
    final_net = build_network(best_x0, best_x1)
    pp.runpp(final_net)
    # 각 버스 상세 정보 출력
    print("=== 최종 버스 상세 결과 (res_bus) ===")
    print(final_net.res_bus)
        # 각 선로(라인)의 전력 흐름 정보 출력
    print("\n=== 최종 선로 전력 흐름 결과 (res_line) ===")
    print(final_net.res_line) # 이 줄 추가
main()
